In [17]:
import MySQLdb
from mysql_cred import mysql_user, mysql_passowrd, mysql_database
from pprint import pprint
import pandas as pd

mysql_ip = '158.85.198.80'
mysql_port = 3306

In [18]:
con = MySQLdb.connect(mysql_ip, mysql_user, mysql_passowrd, mysql_database, port = mysql_port)

In [50]:
def exec_pd(query, printit = False):
    con = MySQLdb.connect(mysql_ip, mysql_user, mysql_passowrd, mysql_database, port = mysql_port)
    cursor = con.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    #pprint(result)
    df = pd.DataFrame.from_records(list(result))
    if printit:
        print df
    return df

def exec_list(query, printit = False):
    con = MySQLdb.connect(mysql_ip, mysql_user, mysql_passowrd, mysql_database, port = mysql_port)
    cursor = con.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    #pprint(result)
    df = list(result)
    if printit:
        print df
    return df

In [36]:
exec_pd("select * from rout_routeinfo limit 5;")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,2015-06-29 00:00:26.629730,35,Pittsburg/Bay Point - SFIA/Millbrae,PITT-SFIA,ROUTE 1,1,PITT,SFIA,None,#ffff33,1,26,"PITT,NCON,CONC,PHIL,WCRK,LAFY,ORIN,ROCK,MCAR,1..."
1,2,2015-06-29 00:00:26.629730,35,Millbrae/SFIA - Pittsburg/Bay Point,SFIA-PITT,ROUTE 2,2,SFIA,PITT,None,#ffff33,1,26,"MLBR,SFIA,SBRN,SSAN,COLM,DALY,BALB,GLEN,24TH,1..."
2,3,2015-06-29 00:00:26.629730,35,Fremont - Richmond,FRMT-RICH,ROUTE 3,3,FRMT,RICH,None,#ff9933,1,18,"FRMT,UCTY,SHAY,HAYW,BAYF,SANL,COLS,FTVL,LAKE,1..."
3,4,2015-06-29 00:00:26.629730,35,Richmond - Fremont,RICH-FRMT,ROUTE 4,4,RICH,FRMT,None,#ff9933,1,18,"RICH,DELN,PLZA,NBRK,DBRK,ASHB,MCAR,19TH,12TH,L..."
4,5,2015-06-29 00:00:26.629730,35,Fremont - Daly City,FRMT-DALY,ROUTE 5,5,FRMT,DALY,None,#339933,0,19,"FRMT,UCTY,SHAY,HAYW,BAYF,SANL,COLS,FTVL,LAKE,W..."


In [37]:
exec_pd("describe rout_routeinfo")

,0,1,2,3,4,5
0,uid,bigint(20),NO,PRI,None,auto_increment
1,fetchtime,varchar(26),YES,,None,
2,sched_num,smallint(6),YES,,None,
3,name,text,YES,,None,
4,abbr,varchar(9),YES,,None,
5,routeid,varchar(8),YES,,None,
6,number,tinyint(4),YES,,None,
7,origin,varchar(4),YES,,None,
8,destination,varchar(4),YES,,None,
9,direction,varchar(5),YES,,None,


In [59]:
st = exec_list('select distinct stations from rout_routeinfo')
no = exec_list('select distinct number from rout_routeinfo')

###list of station abbreviations, routes

In [61]:
sts = []
for s in st:
    s = list(s)
    if len(s) > 0:
        sts += (s[0].split(','))

print list(set(sts))
print [i[0] for i in no]

['SHAY', 'FTVL', 'NCON', 'COLS', 'WCRK', 'WOAK', 'CONC', 'CAST', 'ORIN', 'SFIA', 'WDUB', 'DALY', 'EMBR', 'CIVC', 'SBRN', 'HAYW', 'LAFY', 'DBRK', '12TH', 'PHIL', 'PITT', 'NBRK', 'MONT', 'POWL', 'ROCK', 'SANL', '16TH', 'FRMT', 'BALB', 'DELN', 'MCAR', 'DUBL', '24TH', 'PLZA', 'MLBR', 'GLEN', 'BAYF', 'LAKE', 'ASHB', 'UCTY', '19TH', 'COLM', 'RICH', 'SSAN', 'OAKL']
[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 19, 20]
